# Libraries

## Agentic Related

In [17]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from openai import AsyncOpenAI
from agents.model_settings import ModelSettings

## Exterior

In [12]:
from pydantic import BaseModel
import asyncio
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content

## Interior

In [13]:
from dotenv import load_dotenv
import os
from typing import Dict
from IPython.display import display, Markdown

# Development

## Loading in Gemini and OpenAI Models

In [30]:
# OpenAI
load_dotenv(override=True)
gpt_model = "gpt-4o-mini"

# Gemini
gemini = {
    "base_model": "gemini-2.0-flash",
    "api_key": os.getenv("GEMINI_API_KEY"),
    "base_url": "https://generativelanguage.googleapis.com/v1beta"
}

gemini_client = AsyncOpenAI(base_url=gemini["base_url"], api_key=gemini["api_key"])
gemini_model = OpenAIChatCompletionsModel(model=gemini["base_model"], openai_client=gemini_client)

## Development Summary
* OpenAI Agents SDK includes some hosted tools, these include:\
WebSearchTool --> Allows an agent to search the web.\
FileSearchTool --> Allows retrieving information from OpenAI Vector Stores.\
ComputerTool --> Allows automating computer use tasks. These could include taking screenshots, clicking, searching, or other operations, which would require guardrails to limit certain conditions.

## Search Agent

In [ ]:
# The search agent's instructions parameters values. (these can be changed based on your liking)
search_agent_instructions_params = {
    "min_paragraph_count": 2,
    "max_paragraph_count": 3,
    "min_word_count": 200,
    "max_word_count": 300,
    "tone": "semi-formal"
}

# Search's dedicated instructions
search_agent_instructions = f" You are a research assistant.\
                Given a search term, you search the web for that term and produce a concise summary of the results.\
                The summary must be {search_agent_instructions_params["min_paragraph_count"]}-{search_agent_instructions_params["max_paragraph_count"]} paragraphs and between than {search_agent_instructions_params["min_word_count"]}-{search_agent_instructions_params["max_word_count"]} words.\
                Capture the main points. Write in a {search_agent_instructions_params["tone"]} tone.\
                This will be consumed by someone synthesizig a report, so it's vital you capture the essense and ignore any fluff.\
                Do not include any additional commentary other than the summary itself."

In [47]:
search_agent = Agent(
    name="Search Agent",
    instructions=search_agent_instructions,
    # due to the costs of using the websearchtool, i will use the low settings.
    tools=[WebSearchTool(search_context_size="low")],
    model=gpt_model,
    # search agent is expected to run the tool. 
    model_settings=ModelSettings(tool_choice="required")
)

In [ ]:
sample_agent_prompt = "Nvidia's AI innovations of 2025"

with trace("Search"):
    result = await Runner.run(search_agent, sample_agent_prompt)
display(Markdown(result.final_output))

## Planner Agent
The overall purpose of the planner agent is to produce x searches which will help the search agent when it's going to do it's searching.
No searching is conducted here, however it will be used before the searching, in order help navigate the searching terms before applying the WebSearchTool. 
* Takes in a query, and then taking in x amount of searches surrounded around the query.
* The more queries, the more deeper the research material will be (however this will come at a higher cost)

In [74]:
# Due to costs of SearchTool, I will us 2 for now. 
SEARCHES = 3

In [75]:
planner_agent_instructions = f"You are a helpful research assistant. Given a query, come up with a set of web searches to perform to best answer the query.\
                                Output {SEARCHES} terms to query for."

In [76]:
# WebSearchItem stores the reasoning for behind the search (structured output)
class WebSearchItem(BaseModel):
    reason: str
    "Your reasoning for why this search is important to the query."
    query: str
    "The search term to use for the web search."

# WebSearchPlan stores the list of web searches to perform to best answer the query (structured output)
class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    "A list of web searches to perform to best answer the query."

In [77]:
planner_agent = Agent(
    name="Planner Agent",
    instructions=planner_agent_instructions,
    model=gpt_model,
    output_type=WebSearchPlan
)

In [78]:
with trace("Search"):
    result = await Runner.run(planner_agent, sample_agent_prompt)

## Email Agent
* The function will send an email of the summary of the detailed report to my email (kthonnithodi@gmail.com)
* The sendgrid api has already been pre configured with the api to my email, however in the future I would like to use any email in the parameter.

In [84]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    sg_instance = sendgrid.SendGridAPIClient(api_key=os.environ.get("SENDGRID_API_KEY"))
    from_email = Email("kthonnithodi@gmail.com")
    to_email = To("kthonnithodi@gmail.com")
    context = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, context).get()
    response = sg_instance.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [86]:
email_agent_instructions = (
    "You are able to send a nicely formatted HTML email based on a detailed report.\n"
    "You will be provided with a detailed report. You should use your tool to send one email, providing the report converted into clean, well presented HTML with an appropriate subject line."
)

In [87]:
email_agent = Agent(
    name="Email Agent",
    instructions=email_agent_instructions,
    tools=[send_email],
    model=gpt_model
)

## Writer Agent
* This agent will produce report, given the findings derived from the search agents summary from it's planner agent queries.
* Right now the Agentic Workflow would like the following: Planner Agent --> Search Agent --> Writer Agent --> Email Agent (this would handover)

In [ ]:
writer_agent_instructions_params = {
    "min_paragraph_count": 5,
    "max_paragraph_count": 8,
    "min_word_count": 200,
    "max_word_count": 300,
}

writer_agent_instructions = (
    f"You are a senior researcher tasked with writing a cohesive report for a research query. \
    You will be provided with the original query, and some initial research conducted by a research assistant.\
    You should first come up with an outline for the report that describles the structure and flow of the report.\
    Then, generate the reportand return that as your final output.\
    The final output should be in markdown format, and it should be lengthy and detailed.\
    Aim for around {writer_agent_instructions_params["min_paragraph_count"]} - {writer_agent_instructions_params["max_paragraph_count"]} paragraphs.\
    Furthermore, report should consist of {writer_agent_instructions_params["min_word_count"]} - {writer_agent_instructions_params["max_word_count"]} word count."
)

In [92]:
# The strucuture of the report (summary, then actual report, then follow up questions at the bottom of the report.)
class ReportData(BaseModel):
    short_summary: str
    "A short 2-3 sentence summary of the findings."
    markdown_report: str
    "The final report."
    follow_up_questions: list[str]
    "Suggested topics to research further."

In [93]:
writer_agent = Agent(
    name="Writer Agent",
    instructions=writer_agent_instructions,
    model=gpt_model,
    output_type=ReportData
)

## Functions for planning and execute the search, using planner and search agent

In [ ]:
async def plan_searches(query: str):
    '''
    1. The planner agent will return the output of 3 searches (reason and query) based on a topic.
    2. The output will show a list of the queries (along with respective reasoning) in the form of a WorkSearchPlan.
    '''
    result = await Runner.run(planner_agent, f"Query: {query}")
    return result.final_output

async def execute_websearch(item: WebSearchItem):
    '''
    Use the search agent to run a web search for an instance of the WebSearchItem (one of the queries produced from the plan_searches)
    '''
    message = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, message)

async def execute_websearches(search_plan: WebSearchPlan):
    '''
    1. Collects the search plan from the plan_searches function call (or the search_plan websearchplan variable)
    2. For each query search query in the search plan, create an asyncio task for using a search query. 
    3. This will scour the internet for each one (in parallel, since they are independant tasks).
    4. After creating a co-routine for each, then apply gather for each task in the co-routine to run cocurrently.
    5. The end result should be compilation x amount of searches for each query (the summary report for each query generate by the research assistant).
    '''
    tasks = [asyncio.create_task(execute_websearch(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    return results